Persistence Baseline & Model Comparison

Context

The project currently includes:
	•	a fully reproducible end-to-end pipeline
	•	structural and content validation tests (ISSUE #6 closed)
	•	separate notebooks for each model family

Before selecting a final production model (selected_model.py), it is necessary to compare all ML models against a trivial baseline to properly contextualize performance gains.

⸻

Objective

Establish a persistence baseline to answer the key question:

Does the ML model actually improve over something trivial?

Baseline definition:
ŷ(t) = y(t−1)

This baseline serves as a minimum scientific benchmark.
Where the baseline lives
	•	Notebook: 06_model_comparison.ipynb
	•	❌ Not part of the automated pipeline
	•	❌ Not implemented in models/
	•	❌ Not covered by pytest tests

The baseline is analytical, not production code.
Key rules
	•	Use the same temporal split as ML models
	•	Evaluate on the same test set
	•	Use the same metrics as ML
	•	Do not tune or optimize the baseline

⸻

Metrics to compare
	•	MAE
	•	RMSE
	•	R²

No additional metrics at this stage.

⸻

Outputs to persist (minimum)

Saved under data/results/:
	•	metrics_baseline_vXXX.csv
	•	columns: model, MAE, RMSE, R2
	•	model = "persistence"

Optional:
	•	predictions_baseline_vXXX.csv
	•	date, y_true, y_pred, error

⸻

Central comparison table (core artifact)
model
MAE
RMSE
R2
persistence
…
…
…
random_forest
…
…
…
xgboost
…
…
…
neural_net
…
…
…

Relative improvement vs baseline

Add a derived column:
improvement_vs_baseline_% = (RMSE_baseline - RMSE_model) / RMSE_baseline
Improvements are always computed against the baseline, never between ML models.

⸻

Suggested visualization (optional)
	•	Bar plot of RMSE by model
	•	Baseline clearly highlighted

⸻

Expected conclusion (Markdown)

The notebook should close with an explicit statement such as:

“Model X reduces RMSE by approximately Y% compared to the persistence baseline, indicating that it captures non-trivial temporal and meteorological patterns beyond simple persistence.”

This text will later be reused in:
	•	README
	•	model selection justification
	•	technical interviews / reviews

⸻

Criterion to move to selected_model.py

Proceed only when:
	•	all candidate models have been evaluated
	•	the comparison table is complete
	•	improvement over the baseline is clear and defensible

Only then:
	•	implement models/selected_model.py
	•	freeze the modeling decision

What NOT to do
	•	Do not integrate the baseline into the pipeline
	•	Do not force ML to “win”
	•	Do not add unnecessary complexity
	•	Do not select a model without this comparisonStatus

📌 Bookmark saved — development deferred
To be revisited once all experimentation notebooks are finalized.